In [2]:
pip install dash_bootstrap_components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 1.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


Importing the necessary libraries

In [1]:
import plotly
plotly.__version__
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objs as go
import plotly.offline as pyo
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

Let's read all the necsssary csv files

In [2]:
orders_df = pd.read_csv('cleaned_orders_P.csv')
lineitem_df = pd.read_csv('Line_item.csv')
customer_df = pd.read_csv('customer.csv')
merged_df = pd.read_csv("merged_data_o.csv")
pd.set_option('display.max_rows', None)

Performing calculations for the dashboard

In [3]:
# Convert the specified columns to datetime format
datetime_columns = ['Paid_at', 'Fulfilled_at', 'Created_at', 'Cancelled_at']

for col in datetime_columns:
    orders_df[col] = pd.to_datetime(orders_df[col], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True)

# Verify the datatype conversion by printing the dtypes of these columns
converted_dtypes = orders_df[datetime_columns].dtypes
#print(converted_dtypes)

orders_df['Year_Month'] = orders_df['Paid_at'].dt.to_period('M')
monthly_sales = orders_df.groupby('Year_Month')['Total'].sum().reset_index()

merged_df = pd.merge(customer_df, orders_df, on='Order_ID')

#print(merged_df.head(5))

sales_by_country = merged_df.groupby('Billing_Country')['Total'].sum()

highest_billing_country = sales_by_country.idxmax()
highest_billing_value = sales_by_country.max()
#print(f"The country with the highest billing is {highest_billing_country} with a total sales of {highest_billing_value:.2f}")

# Sort the sales_by_country in descending order and select the top 10
top_10_sales_by_country = sales_by_country.sort_values(ascending=False).head(10)

# Convert to DataFrame for Plotly
top_10_df = top_10_sales_by_country.reset_index()
top_10_df.columns = ['Country', 'Total Sales']

# Group by 'Billing Country' and 'Billing Province' and calculate total sales
sales_by_country_province = merged_df.groupby(['Billing_Country', 'Billing_Province'])['Total'].sum().reset_index()

# Sort the results to identify top regions
sales_by_country_province_sorted = sales_by_country_province.sort_values(by='Total', ascending=False)

# Assuming sales_by_country_province_sorted is already created and sorted
# Explicitly creating a copy of the DataFrame slice
top_10_regions = sales_by_country_province_sorted.head(10).copy()

# Create a new column 'Region' for plotting that combines 'Billing_Country' and 'Billing_Province'
top_10_regions['Region'] = top_10_regions['Billing_Country'] + ' - ' + top_10_regions['Billing_Province']

# Get unique provinces
provinces_list = merged_df['Billing_Province'].unique()

#lineitem type
merged_df_o = pd.read_csv("merged_data_o.csv")
line = pd.read_csv("/Users/sanyuja/plotlysales/Line_item.csv")
import re

service_keywords = ["Shipping Protection", "Sustainability & Coverage", "Onward VIP Protection"]

gift_cards = ["Gift Card", "E-Gift Card"]

product = ["The Gigi", "The Brenna", "The Slide", "The Sandal",
           "The D'orsay", "The Pashionista", "The Miranda", "The Adrianna",
           "The Slingback", "The Pump", "The Bootie", "The Boot", "Sale"]

# Existing regex for general combos
regex_combos = r'(?:\+|-\sSize\s[0-9]|-\sFinal\sSale|-\s[0-9]-[0-9]\.5|-\s[0-9]+(?:-[0-9]+)?\.[0-9]+)'

# Specific regex patterns for items categorized under shoes or accessories within combos
regex_shoes = r'(.+?) - (\d+)-(\d+\.\d+)'
regex_accessories_kit = r'(.+?\bkit\b.+) - (\d+)-(\d+\.\d+)'


def categorize_lineitem(name):
    # First, check for services and gift cards
    if any(service_keyword.lower() in name.lower() for service_keyword in service_keywords):
        return 'Services'
    if 'Gift' in name:
        return 'Gift Cards'
    # Next, check for specific combo patterns to classify as shoes or accessories
    elif re.search(regex_accessories_kit, name, re.IGNORECASE):
        return 'Accessories'
    elif re.search(regex_shoes, name, re.IGNORECASE):
        return 'Shoes'
    # Check general combos with existing regex
    elif re.search(regex_combos, name, re.IGNORECASE):
        return 'Combos'
    # Check for general product mentions
    elif any(pr.lower() in name.lower() for pr in product):
        return 'Shoes'
    else:
        return 'Accessories'  # Default category if none of the above


# Example application (assuming merged_df is already defined)
line['Item_type'] = line['Lineitem_name'].apply(categorize_lineitem)
#print(line[['Lineitem_name', 'Item_type']].head())

# Keep only the specified columns
columns_to_keep = [
    'Order_ID', 'Lineitem_quantity', 'Lineitem_name',
    'Lineitem_price', 'Item_type'
]
line = line[columns_to_keep]
#print(line.columns)

# Optionally, save the DataFrame to a CSV file
# merged_df.to_csv('aa.csv', index=False)
import re

service_keywords = ["Shipping Protection", "Sustainability & Coverage", "Onward VIP Protection"]

gift_cards = ["Gift Card", "E-Gift Card"]

product = ["The Gigi", "The Brenna", "The Slide", "The Sandal",
           "The D'orsay", "The Pashionista", "The Miranda", "The Adrianna",
           "The Slingback", "The Pump", "The Bootie", "The Boot", "Sale"]

# Existing regex for general combos
regex_combos = r'(?:\+|-\sSize\s[0-9]|-\sFinal\sSale|-\s[0-9]-[0-9]\.5|-\s[0-9]+(?:-[0-9]+)?\.[0-9]+)'

# Specific regex patterns for items categorized under shoes or accessories within combos
regex_shoes = r'(.+?) - (\d+)-(\d+\.\d+)'
regex_accessories_kit = r'(.+?\bkit\b.+) - (\d+)-(\d+\.\d+)'


def categorize_lineitem(name):
    # First, check for services and gift cards
    if any(service_keyword.lower() in name.lower() for service_keyword in service_keywords):
        return 'Services'
    if 'Gift' in name:
        return 'Gift Cards'
    # Next, check for specific combo patterns to classify as shoes or accessories
    elif re.search(regex_accessories_kit, name, re.IGNORECASE):
        return 'Accessories'
    elif re.search(regex_shoes, name, re.IGNORECASE):
        return 'Shoes'
    # Check general combos with existing regex
    elif re.search(regex_combos, name, re.IGNORECASE):
        return 'Combos'
    # Check for general product mentions
    elif any(pr.lower() in name.lower() for pr in product):
        return 'Shoes'
    else:
        return 'Accessories'  # Default category if none of the above


# Example application (assuming merged_df is already defined)
line['Item_type'] = line['Lineitem_name'].apply(categorize_lineitem)
#print(line[['Lineitem_name', 'Item_type']].head())

# Keep only the specified columns
columns_to_keep = [
    'Order_ID', 'Lineitem_quantity', 'Lineitem_name',
    'Lineitem_price', 'Item_type'
]
line = line[columns_to_keep]
#print(line.columns)

# Optionally, save the DataFrame to a CSV file
# merged_df.to_csv('aa.csv', index=False)
# Identify unique lineitem_type values
unique_types = line['Item_type'].unique()
#print(unique_types)

#Revenue by quarter

data = pd.read_csv("merged_data.csv", low_memory=False)


column_to_check = 'Financial Status'
data = data.dropna(subset=[column_to_check])


# Convert 'Created at' column to datetime format with dayfirst=True
data['Created at'] = pd.to_datetime(data['Created at'], dayfirst=True)

# Sort the DataFrame by 'Created at'
data = data.sort_values(by='Created at')

# Forward fill missing values in 'Paid at' column based on 'Created at'
data['Paid at'] = data['Paid at'].ffill()

# Verify if any missing values remain in 'Paid at' column
#print("Remaining missing values in 'Paid at' column:", data['Paid at'].isna().sum())

data.drop(columns=[
    'Financial Status', 'Fulfillment Status', 'Currency', 'Shipping Method', 'Lineitem compare at price', 'Lineitem sku',
    'Lineitem requires shipping', 'Lineitem taxable', 'Lineitem fulfillment status', 'Billing Street', 'Billing Address1', 'Billing Address2', 'Billing Company',
    'Billing City', 'Lineitem quantity', 'Lineitem name', 'Lineitem price', 'Billing Zip', 'Billing Province', 'Billing Country', 'Billing Phone',
    'Shipping Name', 'Shipping Street', 'Shipping Address1', 'Shipping Address2', 'Shipping Company', 'Shipping City', 'Shipping Zip',
    'Shipping Province',
    'Shipping Country',
    'Shipping Phone',
    'Notes',
    'Note Attributes',
    'Payment Method',
    'Payment Reference',
    'Vendor',
    'Id',
    'Tags',
    'Risk Level',
    'Source',
    'Lineitem discount',
    'Tax 1 Name',
    'Tax 1 Value',
    'Tax 2 Name',
    'Tax 2 Value',
    'Tax 3 Name',
    'Tax 3 Value',
    'Tax 4 Name',
    'Tax 4 Value',
    'Tax 5 Name',
    'Tax 5 Value',
    'Phone',
    'Receipt Number',
    'Duties',
    'Billing Province Name',
    'Shipping Province Name',
    'Payment ID',
    'Payment Terms Name',
    'Next Payment Due At',
    'Payment References'
], inplace=True)

column_name_mapping = {'Name': 'OrderID'}

data.rename(columns=column_name_mapping, inplace=True)
data.loc[:, 'Revenue'] = data['Total'] - data['Refunded Amount']
desired_order = [
    'OrderID',
    'Billing Name',
    'Email',
    'Paid at',
    'Discount Amount',
    'Subtotal',
    'Shipping',
    'Taxes',
    'Total',
    'Refunded Amount',
    'Revenue',
    'Discount Code',
    'Accepts Marketing'
]
data = data[desired_order]

# data['Paid at'] = pd.to_datetime(data['Paid at'], format='%d-%m-%Y %H:%M')

data['Paid at'] = pd.to_datetime(data['Paid at'], format='%d-%m-%Y %H:%M', errors='coerce')

# Extract the date part from the datetime and overwrite the column with it
data['Paid at'] = data['Paid at'].dt.date

# For Customer Analysis
#CU = data.copy()

# Assuming 'data' is your DataFrame
data = data.sort_values(by=["Email", "Paid at"])

data = data.dropna(subset = ["Email"])
# Create DataFrame
df = pd.DataFrame(data)

# Convert 'Paid at' column to datetime
df['Paid at'] = pd.to_datetime(df['Paid at'])

# Sort DataFrame by 'Email' and 'Paid at'
df.sort_values(by=['Email', 'Paid at'], inplace=True)

# Group by 'Email' and assign order numbers
df['Order Number'] = df.groupby('Email').cumcount() + 1

# Filter out customers who have their first order between October 2023 to Jan 2024
filtered_df = df[~((df['Paid at'] >= '2023-10-01') & (df['Order Number'] == 1))]

# filtered_df
# Count the total number of unique customers who made purchases
total_customers = len(filtered_df['Email'].unique())

# Count the number of customers who made repeat purchases (order number > 1)
repeat_customers = len(filtered_df[filtered_df['Order Number'] > 1]['Email'].unique())

# Calculate retention rate
retention_rate = (repeat_customers / total_customers) * 100

#print("Total Customers:", total_customers)
#print("Repeat Customers:", repeat_customers)
#print("Retention Rate: {:.2f}%".format(retention_rate))

# Extract quarter and year from 'Paid at' column
df['Quarter'] = df['Paid at'].dt.to_period('Q')
#filtered_df['Quarter'] = filtered_df['Paid at'].dt.to_period('Q')

# Group by quarter and count unique customers
customers_per_quarter = df.groupby('Quarter')['Email'].nunique()

total_revenue = df.groupby('Quarter')['Total'].nunique() #nunique() #sum().reset_index()
#total_revenue['Quarter'] = total_revenue['Quarter'].astype(str)

#print(df['Quarter'].head(5))
#print(total_revenue.head(5))
#SUBSET OF ORDERS_DF(selective columns only)

Or1 = orders_df[['Order_ID', 'Email', 'Paid_at', 'Total', 'Refunded_Amount', 'Discount_Amount']]

# Revenue Computation
Or1['Revenue'] = Or1['Total'] - Or1['Refunded_Amount']

Or1 = Or1.sort_values(by=["Revenue", "Paid_at"], ascending=False)

# Extract month and year from 'Paid_at' column
Or1['YearQuarter'] = Or1['Paid_at'].dt.to_period('Q')

# Group by YearQuarter and sum the 'Revenue' column
quarterly_revenue = Or1.groupby('YearQuarter')['Revenue'].sum().reset_index()

# Sort the quarters in chronological order
quarterly_revenue = quarterly_revenue.sort_values(by='YearQuarter')

# Convert Period object to string for plotting
quarterly_revenue['YearQuarter'] = quarterly_revenue['YearQuarter'].astype(str)

#item type dashboard

# Color map for the second section
color_map = {
    'Forest Velvet': '#228B22',  # Forest Green
    'Coal Leather': '#333333',  # Dark grey, close to black
    'Whiskey Leather': '#d29062',  # A warm, medium brown
    'Coal Knit': '#363636',  # Dark grey, slightly lighter than Coal
    'Sand Patent Rhinestone': '#c2b280',  # Light brown, approximation
    'Butter': '#fffacd',  # Light creamy yellow
    'Whiskey Knit': '#d29062',  # Similar to Whiskey Leather
    'White Leather': '#ffffff',
    'Taupe Blush Leather Stud': '#b38b6d',  # Taupe with pinkish hue
    'Coal Leather Stud': '#333333',  # Same as Coal Leather
    'Taupe Blush': '#b38b6d',  # Similar to Taupe Blush Leather Stud
    'Storm': '#71797E',  # Grey with a hint of blue
    'White Satin Lace': '#fffff0',  # Ivory, close to White Satin
    'Storm Suede': '#4F4F4F',  # Greyish-black
    'Sand Braid': '#c2b280',  # Similar to Sand Patent
    'Sand Leather': '#f1c27d',  # Light brown-yellow
    'Storm Knit': '#b0e0e6',  # Powder blue, as requested
    'Whiskey': '#d29062',  # Same as Whiskey Leather
    'Coal': '#333333',  # Coal black
    'Latte Leather': '#c5a883',  # Similar to Latte
    'Sand Raffia': '#c2b280',  # Similar to Sand Braid
    'Walnut': '#773f1a',  # Dark brown
    'Latte': '#c5a883',
    'Walnut Patent': '#773f1a',  # Similar to Walnut
    'Orchid Braid': '#da70d6',  # Similar to Orchid
    'Sand Knit': '#8b4513',  # Brownish, as requested
    'Whiskey Braid': '#d29062',  # Similar to Whiskey
    'Sand': '#f4a460',  # Sandy brown
    'Latte Knit': '#c5a883',  # Same as Latte, with knit pattern implied
    'Coal Patent Bow': '#333333',  # Same as Coal, with a patent finish
    'White Satin': '#fffff0',  # Ivory, close to White Satin Lace
    'Latte Patent': '#c5a883',  # Patent finish of Latte
    'Navy Leather': '#000080',  # Navy blue
    'Clear/Sand ': '#f4a460',  # Sandy brown, clear implied in product type
    'White Crystal Bow': '#ffffff',  # White with crystal implied
    'White Satin Pearl': '#fffff0',  # Ivory, close to White Satin
    'Sand Transparent Rhinestone': '#c2b280',  # Light brown, transparent implied
    'Coal Braid': '#333333',  # Same as Coal
    'Sand Patent': '#f4a460',  # Light brown-yellow, as requested
    'White': '#ffffff',
    'Coal Canvas': '#333333',  # Same as Coal
    'Mango Leather': '#FFC107',  # Bright yellow-orange
    'Latte Braid': '#c5a883',  # Same as Latte
    'Lavender Braid': '#e6e6fa',  # Light purple, similar to Lavender
    'Mango Canvas': '#FFC107',  # Same as Mango Leather
    'Cocoa Patent': '#d2691e',  # Chocolate brown, patent implied
    'Sand Patent Bow': '#f4a460',  # Sandy brown, patent finish
    'Lavender': '#e6e6fa',
    'White Satin Bow': '#fffff0',  # Ivory, satin bow implied
    'Ivory Satin Bow': '#fffff0',  # Ivory, similar to White Satin Bow
    'Orchid': '#da70d6',
    'Mango Elastic': '#FFC107',  # Bright yellow-orange
    'White Canvas': '#ffffff',  # White, canvas texture implied
    'Burgundy Velvet': '#800020',  # Deep red
    'Blue Denim Daisy': '#1f75fe',  # Bright blue, denim implied
    'Matte Gold': '#d4af37',  # Gold, matte finish
    'Coal Patent Coal Rhinestone': '#333333',  # Coal, patent with rhinestone
    'Taupe Blush Suede': '#b38b6d',  # Taupe Blush, suede finish
    'Coal Crystal': '#333333',  # Coal, crystal finish implied
    'Metallic Gold': '#d4af37',  # Bright gold, metallic finish
}
df_line = pd.read_csv("Line_item.csv")
#df_line.head()

# Identify unique lineitem_type values
unique_types = df_line['Item_Type'].unique()
unique_types


# Create a dictionary to hold the DataFrames for each lineitem_type
dfs = {}

for lineitem_type in unique_types:
    # Filter the DataFrame for each unique lineitem_type and store it in the dictionary
    dfs[lineitem_type] = df_line[df_line['Item_Type'] == lineitem_type]

#dfs['Accessories'].head(5)

dfs['Accessories']['Lineitem_name'].count()

df_accessories = dfs['Accessories']

df_acc_block = df_accessories[df_accessories['Lineitem_name'].str.contains('block', case=False)]

# df_acc_block.to_csv('/content/sorted__acc_Line_item.csv')

# Display the first few rows of the filtered DataFrame
#print(df_acc_block.head())

df_acc_stiletto = df_accessories[df_accessories['Lineitem_name'].str.contains('stiletto', case=False)]

# df_acc_stiletto.to_csv('/content/sorted__acc_stiletto.csv')

# Display the first few rows of the filtered DataFrame
#print(df_acc_stiletto.head())

df_combo = dfs['combo']
#df_combo.head()

# Define a regex pattern to capture the Lineitem, Color, Accessories, and Size
pattern = r'(.*) - (.+?) \+ (.+?) - (\d+\.\d+|\d+)'

df_combo[['Lineitem', 'Color', 'Accessories', 'Size']] = df_combo['Lineitem_name'].str.extract(pattern)

#df_combo.head(5)

#df_combo.info()

df_coverage = dfs['Coverage']
#df_coverage.head()

# Regular expression pattern to match the price
pattern = r'\$(\d+\.\d{2})'

# Extract the price and assign it to a new column 'Price'
df_coverage['Price'] = df_coverage['Lineitem_name'].str.extract(pattern)

#print(df_coverage.head())

# Convert both columns to a common type (string) for comparison, in case they are different types
df_coverage['Price'] = df_coverage['Price'].astype(str)
df_coverage['Lineitem_price'] = df_coverage['Lineitem_price'].astype(str)

# Create a new column to indicate if the values in the two columns are identical
df_coverage['is_identical'] = df_coverage['Price'] == df_coverage['Lineitem_price']

# Check if all values are True (identical)
all_identical = df_coverage['is_identical'].all()

# Print the result and the DataFrame for verification
#print(f"All values are identical: {all_identical}")
#print(df_coverage[['Price', 'Lineitem_price', 'is_identical']].head(10))

df_heel_kits = dfs['Heel']
#df_heel_kits.head()

# Filter the DataFrame to only include rows where Item_Type is 'Heel'
df_heel_kits= df_heel_kits[df_heel_kits['Item_Type'] == 'Heel']

# Define a function to categorize the heels based on keywords in 'Lineitem_name'
def categorize_heels(name):
    if 'Block' in name:
        return 'Block Heel'
    elif 'Stiletto' in name:
        return 'Stiletto Heel'
    # Add more conditions as needed for different types
    else:
        return 'Other Type'

# Apply the categorization function to the 'Lineitem_name' column
df_heel_kits['Heel_Category'] = df_heel_kits['Lineitem_name'].apply(categorize_heels)

# Now df_heels includes a new column 'Heel_Category' with the type of heel kit
#df_heel_kits.head()

df_sandals = dfs['Sandals']
#df_sandals.head()

# Here we define two regex patterns for the two different types of line item names
pattern1 = r'(.*) - (.*) - (\d+)'
pattern2 = r'(.*) - (.*) \+ (.*) - (\d+\.?\d*)'

# Function to apply the regular expressions
def extract_info(row):
    if '+' in row['Lineitem_name']:
        matches = re.match(pattern2, row['Lineitem_name'])
        if matches:
            return pd.Series([matches.group(1), matches.group(2), matches.group(3), matches.group(4)],
                             index=['Shoes', 'Color', 'Added_item', 'Size'])
        else:
            return pd.Series([None, None, None, None], index=['Shoes', 'Color', 'Added_item', 'Size'])
    else:
        matches = re.match(pattern1, row['Lineitem_name'])
        if matches:
            return pd.Series([matches.group(1), matches.group(2), None, matches.group(3)],
                             index=['Shoes', 'Color', 'Added_item', 'Size'])
        else:
            return pd.Series([None, None, None, None], index=['Shoes', 'Color', 'Added_item', 'Size'])

# Apply the function to the DataFrame
df_sandals[['Shoes', 'Color', 'Added_item', 'Size']] = df_sandals.apply(extract_info, axis=1)
#df_sandals.head()

subset = df_sandals[['Shoes', 'Color','Added_item','Size']]
#subset.head()

df_sandals['Shoes'].unique()
#print(len(df_sandals['Shoes'].unique()))

#df_sandals.info()
#df_combo.head()

import pandas as pd
from itertools import combinations
from collections import Counter


# Clean the data: Convert all line items to strings and handle NaNs
df_combo['Lineitem'] = df_combo['Lineitem'].astype(str).fillna('Unknown')

# Grouping the data by Order_Id and collecting Lineitems in each order
orders_lineitems = df_combo.groupby('Order_ID')['Lineitem'].apply(list)

# Initialize a counter for lineitem pairs
lineitem_pairs = Counter()

for lineitems in orders_lineitems:
    # Ensure all items in lineitems are strings to avoid the TypeError
    lineitems_str = [str(item) for item in lineitems]
    for pair in combinations(set(lineitems_str), 3):
        sorted_pair = tuple(sorted(pair))
        lineitem_pairs[sorted_pair] += 1

# Convert the lineitem pairs counter to a DataFrame for easier analysis
lineitem_pairs_df = pd.DataFrame(lineitem_pairs.items(), columns=['Lineitem Pair', 'Frequency'])

# Display the top 10 most common lineitem pairs
top_lineitem_pairs = lineitem_pairs_df.sort_values('Frequency', ascending=False).head(10)
#print(top_lineitem_pairs)

# Split 'Lineitem Pair' into three separate columns
lineitem_pairs_df[['Product1', 'Product2', 'Product3']] = pd.DataFrame(lineitem_pairs_df['Lineitem Pair'].tolist(), index=lineitem_pairs_df.index)

# Drop the original 'Lineitem Pair' column
lineitem_pairs_df.drop(columns=['Lineitem Pair'], inplace=True)

# Sort the DataFrame by 'Frequency' to get the top combinations
lineitem_pairs_df = lineitem_pairs_df.sort_values('Frequency', ascending=False)

# Display the modified DataFrame (top 10)
#print(lineitem_pairs_df.head(10))

# Create a new column 'Combination' for plotting
lineitem_pairs_df['Combination'] = lineitem_pairs_df[['Product1', 'Product2', 'Product3']].agg(' | '.join, axis=1)

import pandas as pd
from collections import Counter
import plotly.express as px

# Generate the (lineitem, accessory) pairs and count their frequencies
item_accessory_pairs = Counter(zip(df_combo['Lineitem'], df_combo['Accessories']))

# Initialize an empty list to hold the lineitem, accessory, and frequency
item_accessory_pairs_list = []

for (lineitem, accessory), frequency in item_accessory_pairs.items():
    item_accessory_pairs_list.append({'Lineitem': lineitem, 'Accessories': accessory, 'Frequency': frequency})

# Convert the list of dictionaries to a DataFrame
item_accessory_pairs_df = pd.DataFrame(item_accessory_pairs_list)

# Display the top 10 most common 'Lineitem'-'Accessory' pairs
top_item_accessory_pairs = item_accessory_pairs_df.sort_values('Frequency', ascending=False).head(10)
#print(top_item_accessory_pairs)

df_combo['Lineitem'].unique()

# Assuming 'df_combo' is loaded with relevant 'Lineitem' and 'Color' columns
# Aggregate data to calculate frequency of each color for every lineitem
color_frequency = df_combo.groupby(['Lineitem', 'Color']).size().reset_index(name='Frequency')

# Sort the results to make sure the most frequent colors are listed first for each lineitem
color_frequency_sorted = color_frequency.sort_values(['Lineitem', 'Frequency'], ascending=[True, False])

# Visualize the top N colors for each lineitem
top_n_colors = 5
top_colors_per_lineitem = color_frequency_sorted.groupby('Lineitem').head(top_n_colors)

# Update the top_colors_per_lineitem DataFrame with the corresponding color codes
top_colors_per_lineitem['Color_Code'] = top_colors_per_lineitem['Color'].map(color_map)

# Aggregate data to calculate frequency of each size for every lineitem
size_frequency = df_combo.groupby(['Lineitem', 'Size']).size().reset_index(name='Frequency')

# Sort the results to make sure the most frequent sizes are listed first for each lineitem
size_frequency_sorted = size_frequency.sort_values(['Lineitem', 'Frequency'], ascending=[True, False])

# Visualize the top N sizes for each lineitem
top_n_sizes = 5
top_sizes_per_lineitem = size_frequency_sorted.groupby('Lineitem').head(top_n_sizes)

numeric_sizes = pd.to_numeric(df_combo['Size'], errors='coerce').dropna()


In [4]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

In [5]:

# Define the layout of the dashboard
app.layout = dbc.Container(style={'backgroundColor': '#f0f0f0'}, children=[
    dbc.Row([
        dbc.Col([
            html.H1("Sales Dashboard", style={'textAlign': 'center'})
        ], width=12)
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': country, 'value': country} for country in merged_df['Billing_Country'].unique()],
                value='ALL',
                placeholder="Select a country",
                style={'width': '100%'}
            ),
            dcc.Graph(id='sales-by-country-graph')
        ], width=12)
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='province-dropdown',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': province, 'value': province} for province in provinces_list],
                value='ALL',
                placeholder="Select a province",
                style={'width': '100%'}
            ),
            dcc.Graph(id='sales-by-province-graph')
        ], width=12)
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='payment-dropdown',
                options=[{'label': 'ALL', 'value': 'ALL'}] + [{'label': payment, 'value': payment} for payment in merged_df['Payment_Method'].unique()],
                value='ALL',
                placeholder="Select a payment method",
                style={'width': '100%'}
            ),
            dcc.Graph(id='revenue-by-payment-method-graph')
        ], width=12)
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Input(id='dummy-input', style={'display': 'none'}, value='dummy-value'),
            dcc.Graph(id='revenue-by-item-type-pie-chart')
        ], width=6),
        dbc.Col([
            dcc.Graph(id='marketing-yes-pie-chart')
        ], width=6)
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='orders-by-quarter-graph')
        ], width=12)
    ]),

    # New components for "Item Type Dashboard"
    dbc.Row([
        dbc.Col([
            html.H1("Item Type Dashboard", style={'textAlign': 'center'})
        ], width=12)
    ]),

    dbc.Row([
        dbc.Col([
            html.Div([
                dcc.Markdown('### Select Size'),  # Title for the RangeSlider
                dcc.RangeSlider(
                    id='size-range-slider',
                    min=int(numeric_sizes.min()),
                    max=int(numeric_sizes.max()),
                    value=[int(numeric_sizes.min()), int(numeric_sizes.max())],
                    marks={int(size): str(size)
                           for size in range(int(numeric_sizes.min()), int(numeric_sizes.max()) + 1)},
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ])
        ], width=6),

        dbc.Col([
            html.Div([
                dcc.Markdown('#### Select Line Item'),
                dcc.Dropdown(
                    id='line-item-dropdown',
                    options=[{'label': 'All', 'value': 'All'}] + [{'label': lineitem, 'value': lineitem} for lineitem in df_combo['Lineitem'].unique()],
                    value='All'  # Set default value to 'All'
                )
            ])
        ], width=6)
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='size-revenue-graph', figure={})  # Placeholder, replace with your component
        ], width=6),
        dbc.Col([
            dcc.Graph(id='revenue-graph', figure={})  # Placeholder, replace with your component
        ], width=6),
    ]),

    dbc.Row([
        dbc.Col([
            html.Div([
                dcc.Markdown('#### Select Color'),
                dcc.Dropdown(
                    id='color-dropdown',
                    options=[{'label': 'All', 'value': 'All'}] + [{'label': color, 'value': color} for color in color_map.keys()],
                    value='All'  # Set default value to 'All'
                )
            ])
        ], width=6),

        dbc.Col([
            dcc.Graph(id='color-graph', figure={})  # Placeholder, replace with your component
        ], width=12),
    ]),
])

In [6]:
# Define callback to update graph based on payment method selection
@app.callback(
    dash.dependencies.Output('revenue-by-payment-method-graph', 'figure'),
    [dash.dependencies.Input('payment-dropdown', 'value')]
)
def update_payment_graph(selected_payment_method):
    if selected_payment_method == 'ALL':
        payment_data = merged_df
    else:
        payment_data = merged_df[merged_df['Payment_Method'] == selected_payment_method]

    revenue_by_payment_method = payment_data.groupby('Payment_Method')['Total'].sum().reset_index()
    sorted_revenue = revenue_by_payment_method.sort_values(by='Total', ascending=False)

    # Select only the top 10 payment methods
    top_10_revenue = sorted_revenue.head(10)

    data_p = [
        go.Bar(
            x=top_10_revenue['Payment_Method'],
            y=top_10_revenue['Total']
        )
    ]

    layout = go.Layout(
        title='Revenue by Payment Method (Top 10)',
        xaxis=dict(title='Payment Method', tickangle=45),
        yaxis=dict(title='Revenue'),
        margin=dict(b=150)
    )

    return {'data': data_p, 'layout': layout}


In [7]:
# Define callback to update graph based on dropdown selection
@app.callback(
    dash.dependencies.Output('sales-by-country-graph', 'figure'),
    [dash.dependencies.Input('country-dropdown', 'value')]
)
def update_graph(selected_country):
    if selected_country == 'ALL':
        sorted_top_10_df = top_10_df.sort_values(by='Total Sales', ascending=True)
        data_c = [
            go.Bar(
                y=sorted_top_10_df['Country'],
                x=sorted_top_10_df['Total Sales'],
                orientation='h'
            )
        ]
    else:
        country_data = merged_df[merged_df['Billing_Country'] == selected_country]
        country_sales = country_data.groupby('Billing_Country')['Total'].sum().reset_index()
        data_c = [
            go.Bar(
                y=country_sales['Billing_Country'],
                x=country_sales['Total'],
                orientation='h'
            )
        ]

    layout = go.Layout(
        title='Sales by Country',
        yaxis=dict(title='Country'),
        xaxis=dict(title='Total Sales')
    )

    return {'data': data_c, 'layout': layout}



In [8]:
# Define callback to update graph based on province selection
@app.callback(
    dash.dependencies.Output('sales-by-province-graph', 'figure'),
    [dash.dependencies.Input('province-dropdown', 'value')]
)
def update_province_graph(selected_province):
    if selected_province == 'ALL':
        data_s = [
            go.Bar(
                y=top_10_regions['Region'],
                x=top_10_regions['Total'],
                orientation='h'
            )
        ]
    else:
        province_data = top_10_regions[top_10_regions['Billing_Province'] == selected_province]
        province_sales = province_data.groupby('Region')['Total'].sum().reset_index()
        sorted_province_sales = province_sales.sort_values(by='Total', ascending=False)
        data_s = [
            go.Bar(
                y=sorted_province_sales['Region'],
                x=sorted_province_sales['Total'],
                orientation='h'
            )
        ]

    layout = go.Layout(
        title='Sales by Province',
        yaxis=dict(title='Region'),
        xaxis=dict(title='Total Sales')
    )

    return {'data': data_s, 'layout': layout}


In [9]:
# Define callback to update pie chart based on revenue generated by item type
@app.callback(
    dash.dependencies.Output('revenue-by-item-type-pie-chart', 'figure'),
    [dash.dependencies.Input('dummy-input', 'value')]
)
def update_item_type_pie_chart(dummy_input):
    revenue_by_item_type = line.groupby('Item_type')['Lineitem_price'].sum().reset_index()
    #lineitem_df.groupby('Item_Type')['Lineitem_price'].sum().reset_index()

    data_pc = [
        go.Pie(
            labels=revenue_by_item_type['Item_type'],
            values=revenue_by_item_type['Lineitem_price']
        )
    ]

    layout = go.Layout(
        title='Revenue by Item Type'
    )

    return {'data': data_pc, 'layout': layout}


In [10]:

# Define callback to update pie chart based on percentage of users who say yes to marketing
@app.callback(
    dash.dependencies.Output('marketing-yes-pie-chart', 'figure'),
    [dash.dependencies.Input('dummy-input', 'value')]
)
def update_marketing_yes_pie_chart(dummy_input):
    # Count the number of users who say yes and no to marketing
    marketing_counts = merged_df['Accepts_Marketing'].value_counts()

    # Calculate the percentage of users who say yes and no to marketing
    total_users = marketing_counts.sum()
    yes_percentage = (marketing_counts['yes'] / total_users) * 100
    no_percentage = (marketing_counts['no'] / total_users) * 100

    # Create the pie chart data
    data_ypc = [
        go.Pie(
            labels=['Yes', 'No'],
            values=[yes_percentage, no_percentage]
        )
    ]

    layout = go.Layout(
        title='Percentage of Users Who Say Yes to Marketing'
    )

    return {'data': data_ypc, 'layout': layout}



In [11]:

# Define callback to update graph based on revenue generated per quarter
@app.callback(
    dash.dependencies.Output('orders-by-quarter-graph', 'figure'),
    [dash.dependencies.Input('dummy-input', 'value')]  # Input to trigger callback, you can change this if needed
)
def update_orders_by_quarter(dummy_input):

    data_q = [
        go.Bar(
            x=customers_per_quarter.index.strftime('%Y-Q%q'), #.index.strftime('%Y-Q%q'),
            y=customers_per_quarter.values, #quarterly_revenue['Revenue'], #total_revenue.values
            marker=dict(color='#1f77b4')
        )
    ]

    layout = go.Layout(
        title='Orders placed by Quarter',
        xaxis=dict(title='Quarter'),
        yaxis=dict(title='Count of Orders'),
        hovermode='closest',
        plot_bgcolor='#f0f0f0',
        paper_bgcolor='#f0f0f0'
    )

    return {'data': data_q, 'layout': layout}


In [12]:

# Callback to update the revenue pie chart based on selected line item
@app.callback(
    Output('revenue-graph', 'figure'),
    Input('line-item-dropdown', 'value')
)
def update_revenue_graph(selected_line_item):
    if selected_line_item == 'All':
        filtered_data = df_combo  # Show all line items
    else:
        filtered_data = df_combo[df_combo['Lineitem'] == selected_line_item]

    # Calculate total revenue generated by each line item
    revenue_by_lineitem = filtered_data.groupby('Lineitem').agg({'Lineitem_price': 'sum'}).reset_index()
    revenue_by_lineitem['Tooltip_Text'] = 'Line Item: ' + revenue_by_lineitem['Lineitem'] + '<br>' + \
                                           'Revenue: $' + revenue_by_lineitem['Lineitem_price'].astype(str)

    fig = px.pie(revenue_by_lineitem,
                 values='Lineitem_price',
                 names='Lineitem',
                 title='Total Revenue Generated by Line Item',
                 labels={'Lineitem_price': 'Revenue'},
                 custom_data=['Tooltip_Text'])

    fig.update_traces(hovertemplate='%{customdata[0]}')

    # Highlight the selected slice if a line item is chosen
    if selected_line_item != 'All':
        # Find the index of the selected line item in the dataframe
        selected_index = revenue_by_lineitem[revenue_by_lineitem['Lineitem'] == selected_line_item].index[0]
        # Create a list to store pull values for each slice
        pulls = [0.1 if i == selected_index else 0 for i in range(len(revenue_by_lineitem))]
        fig.update_traces(pull=pulls)

    return fig



In [13]:

# Callback to update the size order count graph based on selected size range
@app.callback(
    Output('size-revenue-graph', 'figure'),
    [Input('size-range-slider', 'value')]
)
def update_size_revenue_graph(size_range):
    # Convert 'Size' column to numeric type
    df_combo['Size'] = pd.to_numeric(df_combo['Size'], errors='coerce')

    # Filter data based on selected size range
    filtered_data = df_combo[(df_combo['Size'] >= size_range[0]) & (df_combo['Size'] <= size_range[1])]

    # Calculate total number of orders for each size
    order_count_by_size = filtered_data.groupby('Size').size().reset_index(name='Order_Count')

    fig = px.bar(order_count_by_size,
                 x='Size',
                 y='Order_Count',
                 title='Order Count by Size',
                 labels={'Order_Count': 'Order Count', 'Size': 'Size'})

    # Add a line graph overlay
    fig.add_trace(go.Scatter(x=order_count_by_size['Size'],
                             y=order_count_by_size['Order_Count'],
                             mode='lines',
                             name='Trends',
                             hoverinfo='skip'))

    return fig



In [14]:

# Callback to update the graph for the 'Colors' tab
@app.callback(
    Output('color-graph', 'figure'),
    Input('color-dropdown', 'value')
)
def update_color_graph(selected_color):
    if selected_color == 'All':
        # Show all colors
        fig = px.bar(color_frequency_sorted,
                     x='Lineitem',
                     y='Frequency',
                     color='Color',
                     color_discrete_map=color_map,
                     title='Most Favorite Colors for Each Lineitem',
                     labels={'Frequency': 'Frequency of Color'},
                     category_orders={"Lineitem": color_frequency_sorted['Lineitem'].unique()})
    else:
        # Filter data based on selected color
        filtered_data = color_frequency_sorted[color_frequency_sorted['Color'] == selected_color]
        fig = px.bar(filtered_data,
                     x='Lineitem',
                     y='Frequency',
                     color='Color',
                     color_discrete_map=color_map,
                     title=f'Most Favorite Colors for Each Lineitem - {selected_color}',
                     labels={'Frequency': 'Frequency of Color'},
                     category_orders={"Lineitem": color_frequency_sorted['Lineitem'].unique()})

    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    return fig



In [18]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8050)